<a href="https://colab.research.google.com/github/MartinPSE/PyTorch_Basic/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 합성곰과 풀링 ( Convolution and Pooling)
---
  

### CNN 으로 MNIST 분류

#### 1.

In [1]:
import torch
import torch.nn as nn

In [12]:
inputs = torch.Tensor(1,1,28,28)

In [13]:
# 천천히 단계별로

conv1 = nn.Conv2d(1, 32, 3, padding = 1)
conv2 = nn.Conv2d(32,64, 3, padding = 1)
pool = nn.MaxPool2d(2)



In [14]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [15]:
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [16]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [17]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [22]:
out.size(0)

1

In [23]:
# 펼치고 --> FullyConnected

out = out.view(out.size(0), -1)
# out --> (1, 64*7*7)

print(out.shape)

fc = nn.Linear(3136, 10) # input_dim --> 64 * 7 * 7 , output_dim = class 갯수
out = fc(out)
print(out.shape)

torch.Size([1, 3136])
torch.Size([1, 10])


#### 2. 실습

In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [53]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [54]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [55]:
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환, 정규화는 따로 안하고
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

In [56]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                     batch_size=batch_size,
                                     shuffle= True,
                                     drop_last = True)

In [62]:
for X, Y in data_loader:
  print(X.shape, Y.shape)
  break

torch.Size([100, 1, 28, 28]) torch.Size([100])


In [68]:
class CNN(nn.Module):

  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size = 3, stride=1 , padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size= 2, stride=2))
    
    self.conv2 = nn.Sequential(
        nn.Conv2d(32,64, kernel_size = 3,stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride=2)
    )
    self.fc = nn.Linear(7 * 7 * 64, 10 , bias=True)

    # wjsrufgkqcmd gkswjddmfh rkwndcl chrlghk
    nn.init.xavier_uniform_(self.fc.weight) # _ 해야 적용되죠?

  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    # 쭉 늘리고 FC
    x = x.view(x.size(0),- 1)
    x = self.fc(x)
    return x


In [69]:
model = CNN().to(device)

In [70]:
# cost function / optimizer

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr= learning_rate)


In [71]:
total_batch = len(data_loader)
print(f'총 배치의 수 : {total_batch}')

총 배치의 수 : 600


In [72]:
for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    prediction = model(X)
    cost = criterion(prediction, Y)
    cost.backward()

    optimizer.step()

    avg_cost += cost / total_batch
  print(f'[ Epoch {epoch + 1:>4}] cost : {avg_cost:>.9}')
    

[ Epoch    1] cost : 0.224709436
[ Epoch    2] cost : 0.0647233948
[ Epoch    3] cost : 0.0475874059
[ Epoch    4] cost : 0.0384270102
[ Epoch    5] cost : 0.0314613841
[ Epoch    6] cost : 0.0258141477
[ Epoch    7] cost : 0.0229033101
[ Epoch    8] cost : 0.0180390459
[ Epoch    9] cost : 0.0166602172
[ Epoch   10] cost : 0.0137007805
[ Epoch   11] cost : 0.0117342155
[ Epoch   12] cost : 0.0102496948
[ Epoch   13] cost : 0.00940447301
[ Epoch   14] cost : 0.00884143822
[ Epoch   15] cost : 0.0064844857


In [78]:
mnist_test

Dataset MNIST
    Number of datapoints: 10000
    Root location: MNIST_data/
    Split: Test
    StandardTransform
Transform: ToTensor()

In [77]:
X_test = mnist_test.test_data.view(len(mnist_test), 1,28,28).float().to(device)
X_test.shape

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


torch.Size([10000, 1, 28, 28])

In [84]:
model()

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)

In [94]:
# test 를 해보자

with torch.no_grad():
  X_test = mnist_test.data.view(len(mnist_test), 1,28,28).float().to(device) # 여기서 왜 한방에 쭊 늘리는걸까요? batch_size 만큼 안넣고
  Y_test = mnist_test.targets.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction,1) == Y_test
  accuracy = correct_prediction.float().mean()
  print(f'Accuracy : {accuracy.item()}')

  

Accuracy : 0.9883000254631042


#### 실습2.
---


In [125]:
# 더 깊은 모델로 CNN을 적용해보자

class CNN2(nn.Module):
  def __init__(self):
    super(CNN2, self).__init__()

    self.conv1 = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size = 3,stride=1, padding = 1),
        nn.ReLU(), 
        nn.MaxPool2d(kernel_size = 2, stride=2 ) # 반으로 줄이기
    )
    self.conv2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size= 3, stride= 1, padding= 1 ),
        nn.ReLU(), 
        nn.MaxPool2d(kernel_size =2 , stride= 2) 
    )
    self.conv3 = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size = 3, stride=1 ,padding=1 ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2 , stride = 2 , padding=1)
    )
    # 차원 계산 --> 
    self.fc1 = nn.Linear( 4 * 4 * 128, 625, bias=True)
    nn.init.xavier_uniform_(self.fc1.weight) # 가중치 초기화
    
    self.layer4 =nn.Sequential(
        self.fc1,
        nn.ReLU(),
        nn.Dropout(0.5)
    )
    self.fc2 = nn.Linear(625, 10 ,bias= True)
    nn.init.xavier_uniform_(self.fc2.weight) # 가중치 초기화

  def forward(self, x ):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)

    x = x.view(x.size(0), -1) # fc 들어가기전에 Flatten 
    x = self.layer4(x)
    x = self.fc2(x)
    return x





In [126]:
model= CNN2().to(device)

In [127]:
# loss / optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [128]:
total_batch = len(data_loader)
print(total_batch)

600


In [129]:
for epoch in range(training_epochs):
  avg_cost = 0
  for X , Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()

    prediction = model(X)
    cost = criterion(prediction, Y)
    cost.backward()

    optimizer.step()

    avg_cost += cost / total_batch
  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
  

    


[Epoch:    1] cost = 0.188857481
[Epoch:    2] cost = 0.0509225689
[Epoch:    3] cost = 0.03520092
[Epoch:    4] cost = 0.0281332638
[Epoch:    5] cost = 0.0234673973
[Epoch:    6] cost = 0.019671781
[Epoch:    7] cost = 0.0165314022
[Epoch:    8] cost = 0.0134264324
[Epoch:    9] cost = 0.0113216089
[Epoch:   10] cost = 0.0111554665
[Epoch:   11] cost = 0.011205147
[Epoch:   12] cost = 0.00892508216
[Epoch:   13] cost = 0.00821020454
[Epoch:   14] cost = 0.00740176393
[Epoch:   15] cost = 0.00769690098


In [134]:
# 테스트
# model.eval() 로 안해도 되는건가요!? 

with torch.no_grad():
  model.eval()

  X_test = mnist_test.data.view(len(mnist_test), 1,28,28).float().to(device)
  Y_test = mnist_test.targets.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print(f'Accuracy :{accuracy.item():.2%}')

Accuracy :98.84%
